In [3]:
import sklearn as sk
import numpy as np
import pandas as pd
import nltk as nl
import string as str
import re

In [8]:
fileName='/Users/yarin/Downloads/hillary-clinton-emails/mailinglist.csv'
with open(fileName) as f:
    lines = f.readlines()

In [9]:
df = pd.read_csv(fileName)

In [14]:
df.columns.values

array(['Unnamed: 0', 'emailId', 'sender', 'senderGender',
       'ExtractedSubject', 'ExtractedBodyText', 'receiver',
       'receiverGender', 'mailLength', 'spaces'], dtype=object)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None,preprocessor = None,stop_words = None,max_features = 10000)
vectorizer3 = CountVectorizer(analyzer = "word", tokenizer = None,preprocessor = None,ngram_range=(1,3),stop_words = None,max_features = 10000)
vectorizer2 = TfidfVectorizer(analyzer = "word", tokenizer = None,lowercase=True,preprocessor = None,ngram_range=(1,3),stop_words = None,max_features = 10000)

In [59]:
porter_stemmer = nl.stem.SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([porter_stemmer.stem(w) for w in analyzer(doc)])


### binary stemmed vectorizer
StemmedBinaryVectorizer = StemmedCountVectorizer(min_df=1, analyzer="word", ngram_range=(1, 3), binary='true')

In [60]:
train_data_features = vectorizer.fit_transform(df['ExtractedBodyText'])
train_data_features2 = StemmedBinaryVectorizer.fit_transform(df['ExtractedBodyText'])

In [61]:
train_data_features = train_data_features.toarray()
train_data_features2 = train_data_features2.toarray()

In [62]:
vocab = vectorizer.get_feature_names()
dist = np.sum(train_data_features, axis=0)
print(train_data_features.shape)

(2266, 10000)


In [66]:
from sklearn.ensemble import RandomForestClassifier

np.random.seed(123)
x = np.random.rand(len(df['ExtractedBodyText'])) < 0.7
train_x = train_data_features[x]
test_x = train_data_features[~x]
train_y = df.loc[x,"senderGender"]
test_y = df.loc[~x,"senderGender"]

forest = RandomForestClassifier(n_estimators = 100) 
model = forest.fit( train_x, train_y )
model.score(test_x,test_y)

0.82848837209302328

In [67]:
from sklearn import svm 
clf = svm.LinearSVC() 
clf.fit(train_x, train_y) 
clf.score(test_x,test_y)

0.81395348837209303

In [72]:
from sklearn import neighbors 
knn = neighbors.KNeighborsClassifier(3) 
knn.fit(train_x, train_y) 
knn.score(test_x,test_y)

0.72674418604651159

ValueError: multiclass-multioutput format is not supported